In [1]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# Load pre-trained model and tokenizer
model_name = "gpt2"
model = GPT2LMHeadModel.from_pretrained(model_name)
tokenizer = GPT2Tokenizer.from_pretrained(model_name)

# Encode a prompt and generate text
prompt = "Once upon a time"
input_ids = tokenizer.encode(prompt, return_tensors="pt")

# Generate text
output = model.generate(input_ids, max_length=50, num_return_sequences=1, no_repeat_ngram_size=2)
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)

print(generated_text)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Once upon a time, the world was a place of great beauty and great danger. The world of the gods was the place where the great gods were born, and where they were to live.

The world that was created was not the same


In [ ]:
db_config_file = 'config.txt'

def load_db_config(filename=db_config_file):
    config = configparser.ConfigParser()
    config.read(filename)
    return config



def connect_to_mysql(config):
    connection = mysql.connector.connect(
        host=config["mysql"]['host'],
        username=config['user'],
        password=config['password']
    )
    return connection


def connect_to_post(config):
    connection = psycopg2.connect(
        host=config['host'],
        user=config['user'],
        password=config['password']
    )
    return connection

#config = load_db_config()
# connection = connect_to_mysql(config)
# connection = connect_to_post(config)

def check_database_existence(project_name):
    db_name = str(project_name)
    config = load_db_config()
    
    if config['db_type'] == 'mysql':
        mysql_connection = connect_to_mysql(config)
        # config['database'] = db_name
        try:
            connect = mysql.connector.connect(**mysql_connection)
            cursor = connect.cursor()
            # Check if the database already exists
            cursor.execute(f"SHOW DATABASES LIKE '{db_name}'")

            existing_database = cursor.fetchall()

            cursor.close()
            connect.close()

            return existing_database
        except mysql.connector.Error as err:
            print(f"Error: {err}")
            return []

    elif config['db_type'] == 'postgresql':
        post_connection = connect_to_post(config)
        # config['database'] = db_name
        try:
            connect = psycopg2.connect(**post_connection)
            cursor = connect.cursor()

            # Check if the database already exists
            cursor.execute("SELECT datname FROM pg_database WHERE datname = %s", (db_name,))
            existing_database = cursor.fetchone()

            if existing_database:
                print(f"Database '{db_name}' already exists.")
                cursor.close()
                connect.close()
                return True
            else:
                # print(f"Database '{db_name}' does not exist.")
                return False

        except psycopg2.Error as err:
            print(f'Error: {err}')
            return False
    else:
        print("Please provide valid database..")